In [43]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass

In [44]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))
# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [45]:
web3.eth.accounts

['0xb6ef75b96D856a6cE0C796be4a7ABfB81E5D600A',
 '0xa2D0B6dbCd04a789da44CCECA9d66B6dfc93A124',
 '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290',
 '0x9cbFF84198C1c3ed898C52Fbc0076Bf008476C4B',
 '0x18a31E094820e4C9C79a073A0927FA72635b18C7',
 '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937',
 '0x37594b8C8aC1D94457C86e30dE793099CE659817',
 '0x6a1ab8E6bb7e9CC3463997FC7F38e3B3855282b5',
 '0xAfd3bB14ff7735B0D9792e5bcc9BB5e758a32ccB',
 '0x8b23fa43919cB11b9CDa04FA45B92aaA0A0Adcd8']

In [46]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [ ]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

In [48]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_address,_loaner,_amount,_loansContractAddress):
    organizationContract.functions.setLoansContractAddress(_loansContractAddress).transact();
    check=organizationContract.functions.createLoan(_address,_loaner,_amount).transact()	
    return check

In [49]:
deleteUser()

In [52]:
def getPendingLoans():
    pendingLoans = userContract.functions.getPendingLoans().buildTransaction({'gas': 70000,'gasPrice': web3.toWei('1', 'gwei'),'from': loanieAddress,'nonce': web3.eth.getTransactionCount(loanieAddress)}) 
    private_key = getpass('Enter your key: ') 
    signed_txn = web3.eth.account.signTransaction({'gas': 70000,'gasPrice': web3.toWei('1', 'gwei'),'from': loanieAddress,'nonce': web3.eth.getTransactionCount(loanieAddress)}, private_key=private_key)
    transaction_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    print('Transaction Block: \n', web3.eth.getTransaction(transaction_hash))
    print('Transaction Receipt: \n', web3.eth.getTransactionReceipt(transaction_hash))
    print('Pending loan ended!')

In [73]:
# Suppose bank wants to create a loan
loanieAddress = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(loanieAddress).call()

Enter the addrses: 0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290


In [74]:
index

1

In [75]:
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if not loanieType:
        #res = organizationContract.functions.createLoan(loanieAddress).call()
        res=True
        print(res)
        choice=input("pending loans Y or N")
        if choice == "Y":
            getPendingLoans()
    else :
        choice=input("create loan Y/N?")
        if choice=="Y":
            if createLoan(web3.eth.accounts[2],web3.eth.accounts[6],1000,loansContractAddress):
                print("Loan created")
else:
    print("This account is not registered in our system.")

True
pending loans Y or NY
Enter your key: ········
Transaction Block: 
 AttributeDict({'hash': HexBytes('0x92662f4e859acb3a1b47f26e69efe7cdf08c00ea52325d631b7bc0abee1f9e91'), 'nonce': 12, 'blockHash': HexBytes('0x4783c8228f67f96557b40a3fb0aaa8086c32106d6ef8ed0da9d73cec59edb67e'), 'blockNumber': 256, 'transactionIndex': 0, 'from': '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290', 'to': None, 'value': 0, 'gas': 70000, 'gasPrice': 1000000000, 'input': '0x', 'v': 27, 'r': HexBytes('0x7ca60b9b462b8727ca7b3878463f6f4bdba8bc07aa6278686b3f07bbef628fcb'), 's': HexBytes('0x2c0661789d00c9baaaf11cdc97c69370c5c1299e389eaa365c3acf76102cae93')})
Transaction Receipt: 
 AttributeDict({'transactionHash': HexBytes('0x92662f4e859acb3a1b47f26e69efe7cdf08c00ea52325d631b7bc0abee1f9e91'), 'transactionIndex': 0, 'blockHash': HexBytes('0x4783c8228f67f96557b40a3fb0aaa8086c32106d6ef8ed0da9d73cec59edb67e'), 'blockNumber': 256, 'from': '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290', 'to': None, 'gasUsed': 53000, 'cumula